 # Introduction to NLP 01
 # Lab 02

## The dataset

In [1]:
!python -m spacy download en_core_web_sm
from datasets import load_dataset_builder, get_dataset_split_names, load_dataset, get_dataset_config_names
ds_builder = load_dataset_builder("imdb")

display(ds_builder.info.description, ds_builder.info.features)

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.6 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/home/djulo/.local/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'Large Movie Review Dataset.\nThis is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.'

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

 ### 1. How many splits does the dataset has?

In [2]:
splits = get_dataset_split_names("imdb")

display(splits)
print(f"We have {len(splits)} splits ") # We have three splits

['train', 'test', 'unsupervised']

We have 3 splits 


### 2. How big are these splits ?

In [3]:
# Load the dataset
dataset = load_dataset("imdb")
dataset

Found cached dataset imdb (/home/djulo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|███████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 576.25it/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

The train and test split contain 25000 rows each. While the unsupervised split has 50000 rows.

### 3. What is the proportion of each class on the supervised splits?

The proportion of each class on the supervised splits is **1/2**.

# Naive Bayes classifier
### 1. Preprocessing

In [4]:
# Take the training part of the dataset
train_data = load_dataset("imdb", split="train")

Found cached dataset imdb (/home/djulo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [16]:
# Take a look at the text
text = train_data["text"]
text[0:2]

['I rent I be curious yellow from my video store because of all the controversy that surround it when it be first release in 1967 I also hear that at first it be seize by u s custom if it ever try to enter this country therefore be a fan of film consider controversial I really have to see this for myself the plot be center around a young swedish drama student name lena who want to learn everything she can about life in particular she want to focus her attention to make some sort of documentary on what the average swede think about certain political issue such as the vietnam war and race issue in the united states in between ask politician and ordinary denizen of stockholm about their opinion on politic she have sex with her drama teacher classmate and married man what kill I about I be curious yellow be that 40 year ago this be consider pornographic really the sex and nudity scene be few and far between even then it s not shoot like some cheaply make porno while my countryman mind find

In [17]:
from string import punctuation
import re

def preprocess(dataset: dict) -> dict:
    '''Take a dataset, return the dataset preprocessed'''
    
    # Convert the text to lowercase and remove <br>
    dataset['text'] = dataset['text'].lower().replace('<br /><br />', '')
    
    # Replace punctuation with spaces
    dataset['text'] = re.sub('['+punctuation+']', ' ', dataset['text'])
    
    # Replace multiple with a single space
    dataset['text'] = " ".join(dataset['text'].split())
    
    return dataset

# Apply the preprocessing function to the dataset
train_data = train_data.map(preprocess)

# Take a look at the preprocessed text
train_data['text'][0:2]

['i rent i be curious yellow from my video store because of all the controversy that surround it when it be first release in 1967 i also hear that at first it be seize by u s custom if it ever try to enter this country therefore be a fan of film consider controversial i really have to see this for myself the plot be center around a young swedish drama student name lena who want to learn everything she can about life in particular she want to focus her attention to make some sort of documentary on what the average swede think about certain political issue such as the vietnam war and race issue in the united states in between ask politician and ordinary denizen of stockholm about their opinion on politic she have sex with her drama teacher classmate and married man what kill i about i be curious yellow be that 40 year ago this be consider pornographic really the sex and nudity scene be few and far between even then it s not shoot like some cheaply make porno while my countryman mind find

In [7]:
# Take the split of test.
test_data = load_dataset("imdb", split="test")

# Apply preprocessing to the test dataset
test_data = test_data.map(preprocess)

Found cached dataset imdb (/home/djulo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /home/djulo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-40be9da7ef59e51c.arrow


# 2. Implement own naive Bayes classifier from scratch

In [18]:
import numpy as np
import math

from collections import Counter

class MultinomialNaiveBayesClassifier:
    '''Define Multinomial Naive Bayes Classifier class'''

    def __init__(self) -> None:
        '''Init the logprior, loglikehood and vocabulary for the model'''
        self.logprior = None
        self.loglikelihood = None
        self.vocabulary = None
        self.classes = [0, 1]

    def train_naive_bayes(self, dataset: dict) -> None:
        '''Implementation of the Train Naive Bayes algorithm
            Take a dataset and the set of Class C,
            Define the logprior, loglikehood and the vocabulary
        '''

        # D is the list of all paragraphs
        documents = dataset['text']

        # Number of documents in all the dataset (document = paragraph)
        n_doc = len(dataset)

        # Initialize logprior as dictionnary
        self.logprior = {}

        # Initialize bigdoc as dictionnary
        # bigdoc contain 2 lists of paragraphs depending on their class
        bigdoc = {}

        # Vocabulary is the union of all unique words
        self.vocabulary = set()
        for document in documents:
            self.vocabulary.update(document.split())
        self.vocabulary = list(self.vocabulary)

        # Initialize loglikelihood as dictionnary
        self.loglikelihood = {c: {} for c in self.classes}
        for c in self.classes:
            bigdoc[c] = []

            # Number of documents of class c (document = paragraph)
            n_class = dataset['label'].count(c)

            # Define logprior for class c
            self.logprior[c] = math.log((n_class / n_doc))

            # For each row of the Dataset
            for i in range(len(dataset)):
                if dataset[i]['label'] == c:
                    # Append the document
                    bigdoc[c].append(dataset[i]['text'])

            # Return dictionnay with word as keys and occurence of the word as value.
            word_counts = Counter(' '.join(bigdoc[c]).split(' '))
            total_word_count = sum(word_counts.values())
            for word in word_counts:
                self.loglikelihood[word, c] = math.log((word_counts[word] + 1) / (total_word_count + len(self.vocabulary)))

    def predict(self, testdoc: str) -> int:
        '''Implementation of the test naive Bayes algorithm
            Take a testing document testdoc and the set of class C
            Return a prediction, 0 for negative, 1 for positive'''
        
        sum = {}
        for c in self.classes:
            sum[c] = self.logprior[c]
            for word in testdoc.split(' '):
                if word in self.vocabulary and (word, c) in self.loglikelihood:
                    sum[c] += self.loglikelihood[word, c]
        return max(sum, key=sum.get)
    
    
    
    # Define a function to apply the predict method to a single example
    def predict_all(self, dataset: dict) -> int:
        '''Apply Predict method to an entire dataset
            return a list of predictions'''
        return self.predict(dataset['text'])


# Define our model as a Multinomial Naive Bayes Classifier
model = MultinomialNaiveBayesClassifier()

# Train the model
model.train_naive_bayes(train_data)

#predictions = list(map(model.predict_example, train_data))
#predictions

# We load the predictions result because it takes too much time
with open('predictions_save', 'r') as f:
    content = f.read()
    prediction = content.split(', ')
    
    # convert str to int
    prediction = list(map(int, prediction))
prediction[0:10]

[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]

# 3. Implement a naive Bayes classifier using scikit-learn

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()

# Define y_train as the output y
y_train = train_data['label']

# Apply transformation to the train and test datasets
X_train = vectorizer.fit_transform(train_data['text'])
X_test = vectorizer.transform(test_data['text'])

# Define Multinomial naive bayes model
mnb = MultinomialNB()
mnb.fit(X_train,y_train)

# Predictions of the classifier
mnb_prediction = mnb.predict(X_test)

print(mnb_prediction)

[0 0 0 ... 1 0 0]


# 4. Accuracy

In [10]:
from sklearn.metrics import accuracy_score

# Accuracy score with the scikit-learn implementation
accuracy_sklearn = accuracy_score(mnb_prediction, test_data['label'])
print('Sklearn implementation accuracy: ',accuracy_sklearn)

# Accuracy score with our implentation from scratch
accuracy_scratch = np.sum([prediction[i] == test_data[i]['label'] for i in range(len(prediction))]) / len(prediction)
print('Our implementation accuracy: ',accuracy_scratch)

Sklearn implementation accuracy:  0.8144
Our implementation accuracy:  0.68268


# 5. Why the scikit-learn implementation give better results ?

The main reason why the scikit-learn implementation give better results is the ability to define hyper-parameters:
    
1. alpha: Additive (Laplace/Lidstone) smoothing parameter (set alpha=0 and force_alpha=True, for no smoothing).
2. force_alpha: If False and alpha is less than 1e-10, it will set alpha to 1e-10. If True, alpha will remain unchanged. This may cause numerical errors if alpha is too close to 0.
3. fit_prior: Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
4. class_prior: Prior probabilities of the classes. If specified, the priors are not adjusted according to the data.

    
By playing with those hyper-parameters and by choosing the right ones regarding our dataset we can obtain better result.

# 6. Why is accuracy a sufficient measure of evaluation here?

In this model, accuracy is a sufficient measure of evaluation because we are not dealing with imbalanced classes and because here the cost of false positives and false negatives is considered equivalent.

In these other cases, other metrics like precision, recall, F1 score, or AUC-ROC could have been more appropriate for evaluating the model's performance.

# 7. Explaining why the model failed.


In [11]:
# Search for a failed prediction
failed_predictions_indexes = [i for i in range(len(mnb_prediction)) if mnb_prediction[i] != test_data[i]['label']]

# Take a look at the 2 first documents where the prediction has failed
test_data[failed_predictions_indexes[:2]]['text']

['blind date columbia pictures 1934 was a decent film but i have a few issues with this film first of all i don t fault the actors in this film at all but more or less i have a problem with the script also i understand that this film was made in the 1930 s and people were looking to escape reality but the script made ann sothern s character look weak she kept going back and forth between suitors and i felt as though she should have stayed with paul kelly s character in the end he truly did care about her and her family and would have done anything for her and he did by giving her up in the end to fickle neil hamilton who in my opinion was only out for a good time paul kelly s character although a workaholic was a man of integrity and truly loved kitty ann sothern as opposed to neil hamilton while he did like her a lot i didn t see the depth of love that he had for her character the production values were great but the script could have used a little work',
 'ben rupert grint is a deepl

We think that the predictions failed for these two examples because there are in these texts the presence of many positive and negative words at the same time. Since the naive Bayes classifier is a naive model, it ignores the position of the words and only takes into account their existence in the text, and also the conditional independence assumption who say that the probabilities P(fi|c) are independent.

Those naive assumptions led to a prediction error.

# Bonus

# Stemming and Lemmatization

### 1.
We choose the lemmatization

In [12]:
import spacy

# Loading the small English model
nlp = spacy.load("en_core_web_sm")

def preprocess_lemming(dataset: dict) -> dict:
    '''Apply lemming to the all dataset given in parameter'''
    
    dataset['text'] = lemming(dataset['text'])
    return dataset

def lemming(text: str) -> str:
    '''Apply lemming on a document'''
    
    re_word = re.compile(r"^\w+$")
    lemmas = [token.lemma_ for token in nlp(text) if re_word.match(token.text)]
    return " ".join(lemmas)
   
# Apply lemming to the training dataset
train_data = train_data.map(preprocess_lemming)    
train_data['text'][0]

Loading cached processed dataset at /home/djulo/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-7469e200be2cf060.arrow


'I rent I be curious yellow from my video store because of all the controversy that surround it when it be first release in 1967 I also hear that at first it be seize by u s custom if it ever try to enter this country therefore be a fan of film consider controversial I really have to see this for myself the plot be center around a young swedish drama student name lena who want to learn everything she can about life in particular she want to focus her attention to make some sort of documentary on what the average swede think about certain political issue such as the vietnam war and race issue in the united states in between ask politician and ordinary denizen of stockholm about their opinion on politic she have sex with her drama teacher classmate and married man what kill I about I be curious yellow be that 40 year ago this be consider pornographic really the sex and nudity scene be few and far between even then it s not shoot like some cheaply make porno while my countryman mind find 

### 2.
Train again with the lemmatization

In [13]:
# Apply lemmatization to the test data
test_data = test_data.map(preprocess_lemming)    

In [14]:
# Define y_train as the output y
y_train = train_data['label']

# Apply transformoration to the train and test dataset
X_train = vectorizer.fit_transform(train_data['text'])
X_test = vectorizer.transform(test_data['text'])

# Define Multinomial naive bayes model
mnb = MultinomialNB()
mnb.fit(X_train,y_train)

# predictions of the classifier
mnb_prediction = mnb.predict(X_test)

print(mnb_prediction)

[0 0 0 ... 1 0 0]


### 3.

In [15]:
# accuracy score with the scikit-learn implementation
accuracy_sklearn = accuracy_score(mnb_prediction, test_data['label'])
print(accuracy_sklearn)

0.80908


We can see that the accuracy has decreased after the lemmatization of the dataset text.

We can explain that by the fact that we apply lemmatization without stemming and therefore we could have lost the meaning of certain words.